In [ ]:
import os
import sys
import numpy as np
import warnings
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.modeling import models, fitting
from astropy.time import Time
import math
import glob
import matplotlib.pyplot as plt
import configparser
import csv
from dotenv import load_dotenv
import datetime
import logging
import pandas as pd
%matplotlib inline
from astropy import constants as const

In [ ]:
from modules.radial_velocity.src.alg import RadialVelocityAlg
from modules.radial_velocity.src.alg_rv_init import RadialVelocityAlgInit
load_dotenv()
TEST_DIR = os.getenv('KPFPIPE_TEST_DATA') 
print('TEST_DIR:', TEST_DIR)
FIT_G = fitting.LevMarLSQFitter()
LIGHT_SPEED = 299792.458   # light speed in km/s
class DotDict(dict):
    pass
MODULE_DIR = '../../modules/radial_velocity/'
reweight_method = 'ccf_max'

In [ ]:
MJD_TO_JD = 2400000.5
class RadialVelocityStats:
    """ This module defines class ' RadialVelocityStats' and methods to do statistic analysis on radial velocity
    results. (this is currently for radial velocity development and testing only).

    Attributes:
         rv_result_set (list): A container storing radial velocity result from fits of level 1 data.
         total_set (int): Total elements in `rv_result_set`.
    """

    def __init__(self, obs_rv_results: list = None):
        self.rv_result_set = list() if obs_rv_results is None else obs_rv_results.copy()
        self.total_set = 0 if obs_rv_results is None else len(obs_rv_results)

    def get_collection(self):
        return self.rv_result_set, self.total_set

    def add_data(self, ccf_rv: float, obj_jd: float):
        self.rv_result_set.append({'jd': obj_jd, 'mean_rv': ccf_rv})
        self.total_set = len(self.rv_result_set)
        return self.rv_result_set, self.total_set

    def analyze_multiple_ccfs(self, ref_date=None):
        """ Statistic analysis on radial velocity numbers of multiple observation resulted by `RadialVelocityAlg`.

        Args:
            ref_date (str, optional): Reference time in the form Julian date format.  Defaults to None.

        Returns:
            dict: Analysis data.

        """
        obs_rvs, total_obs = self.get_collection()
        jd_list = np.array([obs_rv['jd'] for obs_rv in obs_rvs])
        if ref_date is None:
            ref_jd = self.get_start_day(jd_list)
        else:
            ref_jd = Time(ref_date, format='isot', scale='utc').jd
        rv_stats = dict()
        rv_stats['start_jd'] = ref_jd
        rv_stats['hour'] = (jd_list-ref_jd) * 24.0
        rv_stats['day'] = (jd_list-ref_jd)
        rv_stats['values'] = np.array([obs_rv['mean_rv'] for obs_rv in obs_rvs])
        rv_stats['mean'] = np.mean(rv_stats['values'])
        rv_stats['sigma'] = np.std(rv_stats['values'] - rv_stats['mean'])

        return rv_stats

    @staticmethod
    def get_start_day(jd_list: np.ndarray):
        min_jd = np.amin(jd_list)
        day_part = math.floor(min_jd - MJD_TO_JD)
        return MJD_TO_JD+day_part


In [ ]:
def plot_gaussian_on_curve(g_curve, curve_x, curve_y, order=None, title=None, ref_curve=None, ref_gaussian=None):
    plt.figure(figsize=(10,10))
    
    label_curve = "rv on order " + str(order) if order is not None else 'after reweighting'
    plt.plot(curve_x, curve_y, 'ko', label=label_curve)
    if ref_curve is not None:
        plt.plot(curve_x, ref_curve, 'mo', label="before reweighting"  )
        plt.plot(curve_x, ref_gaussian(curve_x), label='Gaussian w/ mean:'+str("{0:.6f}".format(ref_gaussian.mean.value)))
    plt.plot(curve_x, g_curve(curve_x), label='Gaussian w/ mean:'+str("{0:.6f}".format(g_curve.mean.value)))
                                                      
    plt.legend(loc="lower right", prop={'size': 12})
    plt.title(title) if title is not None else None
    plt.show()

In [ ]:
import math
dev = 'dev'
def plot_velocity_time(rv_info, title, label, color, time_unit='hrs', savefig=None):
    k1 = list(rv_info.keys())[0]
    
    total_rv = np.size(rv_info[k1]['values'])

    if time_unit == 'hrs':
        rv_delta_time = rv_info[k1]['hour'] 
    else:
        rv_delta_time = rv_info[k1]['day']
        
    plt.figure(figsize=(10,12))     
    s = 100
    ymax = -10000
    ymin = 10000
    for k in rv_info.keys():
        rv_offset = (rv_info[k]['values'] - rv_info[k]['mean']) * 1000.0
        ymax = max(np.amax(rv_offset), ymax)
        ymin = min(np.amin(rv_offset), ymin)
        rv_sigma = rv_info[k]['sigma']*1000.0
        plt.scatter(rv_delta_time, rv_offset, s, c=color[k], edgecolors='b', 
                    label=label[k] + r' $\sigma = $'+ "{:0.6f}".format(rv_sigma)+' m/s')
        

    plt.legend(loc="upper right", prop={'size':12})
    plt.xlabel('Times ['+time_unit+']')
    plt.ylabel('RV [m/s]')

    ymax = math.ceil(ymax)+1
    ymin = math.floor(ymin)-1
    xmin = math.ceil(np.amin(rv_delta_time))
    xmax = math.floor(np.amax(rv_delta_time))
    if time_unit == 'hrs':
        plt.xlim((xmin-1, xmax+1))
    else:
        plt.xlim((xmin-20, xmax+20))
    if title is not None:
        plt.title(title)
    
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()    
    

In [ ]:
def fit_ccf(result_ccf, total_order, velocity_cut=100.0):
    rv_guess = -0.5
    row_for_analysis = np.arange(1, total_order, dtype=int)
    result_ccf[total_order + 2, :] = np.nansum(result_ccf[row_for_analysis, :], axis=0)
    g_init = models.Gaussian1D(amplitude=-1e7, mean=rv_guess, stddev=5.0)
    velocities = result_ccf[total_order + 1, :]
    ccf = result_ccf[total_order + 2, :]
    i_cut = (velocities >= rv_guess - velocity_cut) & (velocities <= rv_guess + velocity_cut)
    g_x = velocities[i_cut]
    g_y = ccf[i_cut] - np.nanmedian(ccf)
    gaussian_fit = FIT_G(g_init, g_x, g_y)
    return gaussian_fit, gaussian_fit.mean.value, g_x, g_y

# Plot level 1 flux vs. orders 

In [ ]:
# plot L1 results of selected order
def plot_level1_data(lev1_data, selected_order, check_order=False, title=None):
    colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'coral', 'purple', 'orange']
    n_colors = len(colors)
    plt.figure(figsize=(18,6))   
    point_per_order = 1
    s_point = 0
    c_idx = 0
    for ord in selected_order:
        v_order = lev1_data[ord, :]
        max_v = np.nanpercentile(v_order, 95)
        p_list = np.where(v_order <= max_v)[0]
        # n_points = np.size(v_order)
        # p_list = np.arange(0, n_points, point_per_order, dtype=int)
        y_data = v_order[p_list]
        x_data = np.arange(s_point, s_point+np.size(y_data), dtype=int)
        nan_idx = np.where(np.isnan(y_data))[0]
        if np.size(nan_idx) > 0:
            y_data[nan_idx] = 0.0
        neg_idx = np.where(y_data < 0)[0]
        if np.size(neg_idx) > 0:
            y_data[neg_idx] = 0.0
        if check_order:
            import pdb;pdb.set_trace()
        plt.plot(x_data, y_data, c = colors[c_idx%n_colors])
        s_point += np.size(x_data) - 1  
        c_idx += 1
    if title is not None:
        plt.title(title)
    plt.show()

In [ ]:
spectrum_set = TEST_DIR + '/NEIDdata/HD127334/L2/neidL2_*.fits'
ccf_idx = -1

s_idx = 0
e_idx = 116
order_diff = 0
Files = sorted(glob.glob(spectrum_set))

for f_idx in range(len(Files)):
    f = Files[f_idx]
    hdulist = fits.open(f)
    lev1_data = hdulist[1].data
    max_v = np.max(lev1_data)
    max_pos = np.where(lev1_data == max_v)
    print('file: ', f, 'max: ', max_v, 'max_loc: ', max_pos)
    # import pdb;pdb.set_trace()
    plot_level1_data(lev1_data, np.arange(s_idx, e_idx+1, dtype=int), title='level 1 for '+f)

## RV computation on NEID L1 data and results from Optimal Extraction module

In [ ]:
# original NEID level 2 data
neid_L1_L2_dir = TEST_DIR + '/NEIDdata/HD127334/L2/neidL2'
neid_L2_files =  sorted(glob.glob(neid_L1_L2_dir + '*.fits'))

print('\n'.join(neid_L2_files))

lev2_result_dir = '../../test_results/neid_hd127334/'
lev2_result_files = sorted(glob.glob(lev2_result_dir + 'neidL1*_L2.fits'))

outfolder = MODULE_DIR+'results/NEID_HD127334/'

s_order=0
e_order=116
order_diff = 0
s_x_pos = 600

# import pdb;pdb.set_trace()
print('\n'.join(lev2_result_files))
total_file = len(lev2_result_files)

"""
config_file = MODULE_DIR + 'configs/default_recipe_neid_hd127334.cfg'
config = configparser.ConfigParser()
config.read(config_file)
logger = start_logger("OrderTraceAlg", config_file)

rv_init = RadialVelocityAlgInit(config, logger)
init_result = rv_init.start(print_debug='')
"""
rv_dev_info = RadialVelocityStats()
rv_dev_reweight_info = RadialVelocityStats()
rv_lev2_info = RadialVelocityStats()
rv_lev2_reweight_info = RadialVelocityStats()


# Pick the template and do reweighting on NEID/developed level2 result

In [ ]:
allcpp = []

for f in range(len(lev2_result_files)):
    hdul = fits.open(lev2_result_files[f])
    ccf = pd.DataFrame(hdul[12].data).values           # a Table HDU
    ccf_max = np.nanmax([np.nanpercentile(ccf[od, :], 95) for od in range(s_order, e_order+1)])
    allcpp.append(ccf_max)
    # plt.figure(figsize=(12,6))
    # plt.plot(np.transpose(ccf[s_order:e_order+1, :]))
    # plt.title(lev2_result_files[f])
    # plt.show()


index = np.where(allcpp == np.max(allcpp))
index = np.squeeze(index[0])
print(index, lev2_result_files[index])
template_file_hdulist = fits.open(lev2_result_files[index])
template_ccf = pd.DataFrame(template_file_hdulist[12].data).values
ratio_file = MODULE_DIR + 'results/NEID_HD127334/hd127334_ratio.csv'
rw_ratio_df = RadialVelocityAlg.make_reweighting_ratio_table(template_ccf, s_order, e_order, reweight_method,
                                                             max_ratio=1.0, output_csv=ratio_file)
# print(rw_ratio_df.values)

allcpp_lev2 = []
for f in range(len(neid_L2_files)):
    hdul = fits.open(neid_L2_files[f])
    ccf = hdul[12].data                               # an Image HDU
    allcpp_lev2.append(np.max(ccf[s_order:e_order+1, :]))
    # plt.figure(figsize=(12,6))
    # plt.plot(np.transpose(ccf[s_order:e_order+1, :]))
    # plt.title(neid_L2_files[f])
    # plt.show()
             
index = np.where(allcpp_lev2 == np.max(allcpp_lev2))   
index = np.squeeze(index[0])
print(index, neid_L2_files[index])
template_neid_lev2_hdulist = fits.open(neid_L2_files[index])
template_neid_ccf = template_neid_lev2_hdulist[12].data
rw_ratio_lev2_df = RadialVelocityAlg.make_reweighting_ratio_table(template_neid_ccf, s_order, e_order, 
                                                                  reweight_method, max_ratio=1.0)
# print(rw_ratio_lev2_df.values)


# reweight on calculated ccf and NEID ccf

In [ ]:
all_ccf = []
total_order = e_order-s_order+1
rv_total = np.zeros((len(lev2_result_files), 164))
after_reweighting_total = np.zeros((len(lev2_result_files), 164))
for f in range(len(lev2_result_files)):
    
    # reweight NEID level 2 ccf
    print(neid_L2_files[f])
    neid_L2_file_hdulist = fits.open(neid_L2_files[f])
    L1_time = neid_L2_file_hdulist[12].header['CCFJDSUM']
    rv_guess = neid_L2_file_hdulist[0].header['QRV']
    ny, nx = np.shape(neid_L2_file_hdulist[12].data)
    
    
    # add info from L2 data
    simplerv = neid_L2_file_hdulist[12].header['CCFRVSUM']
    rv_lev2_info.add_data(simplerv, L1_time)
    
    neid_L2_ccf = np.zeros((total_order+3, nx))
    neid_L2_ccf[0:ny, :] = neid_L2_file_hdulist[12].data
    start_rv = neid_L2_file_hdulist[12].header['CCFSTART']
    step_size = neid_L2_file_hdulist[12].header['CCFSTEP']
    total_step = nx;
    neid_L2_ccf[total_order+1, :] = np.arange(-total_step/2, total_step/2, dtype=int)*step_size + 0.0
    
    reweighted_lev2_ccf = RadialVelocityAlg.reweight_ccf(neid_L2_ccf, total_order, rw_ratio_lev2_df.values, 
                                                    reweight_method, do_analysis=True)

    # import pdb;pdb.set_trace()
    # rw_lev2_ccf_fit, rw_lev2_ccf_mean, n_x, n_y = RadialVelocityAlg.fit_ccf(reweighted_lev2_ccf[total_order+2, :],
    #                                                                        rv_guess, 
    #                                                                        neid_L2_ccf[total_order+1, :])
    reweighted_lev2_ccf[total_order+1, :] = neid_L2_ccf[total_order+1, :]
    rw_lev2_ccf_fit, rw_lev2_ccf_mean, n_x, n_y = fit_ccf(reweighted_lev2_ccf, total_order)
    rv_lev2_reweight_info.add_data(rw_lev2_ccf_mean, L1_time) 
    
    # reweight calculated L2 data
    print(lev2_result_files[f])
    file_hdulist = fits.open(lev2_result_files[f])
    ccf_data = pd.DataFrame(file_hdulist[12].data).values
    
    rv_ccf_fit, rv_ccf_mean, n_x, n_y = fit_ccf(ccf_data, total_order)
    
    # add info from calculated L2 data
    rv_dev_info.add_data(rv_ccf_mean, L1_time)
    rv_total[f, :] = ccf_data[total_order+2, :]

    # add info from weighted L2 data
    reweighted_ccf = RadialVelocityAlg.reweight_ccf(ccf_data, total_order, rw_ratio_df.values, 
                                        reweight_method, do_analysis=True)
    reweighted_ccf[total_order+1, :] = ccf_data[total_order+1, :]
    after_reweighting_total[f, :] = reweighted_ccf[total_order+2, :]
    #ccf_fit, ccf_mean, n_x, n_y = RadialVelocityAlg.fit_ccf(reweighted_ccf[total_order+2, :], rv_guess, 
    #                                                        ccf_data[total_order+1, :])
    ccf_fit, ccf_mean, n_x, n_y = fit_ccf(reweighted_ccf, total_order)
    rv_dev_reweight_info.add_data(ccf_mean, L1_time)
    
    # before_ccf_fit, before_mean, before_x, before_y = RadialVelocityAlg.fit_ccf(ccf_data[total_order+2, :], rv_guess, 
    #                                                                            ccf_data[total_order+1, :])
    before_ccf_fit, before_mean, before_x, before_y = fit_ccf(ccf_data, total_order)
    
    plot_gaussian_on_curve(ccf_fit, n_x, n_y, ref_curve=before_y, ref_gaussian=before_ccf_fit)
    
    # plt.figure(figsize=(18,6))
    # plt.subplot(1, 2, 1)
    # plt.plot(np.transpose(rv_total))
    # plt.subplot(1, 2, 2)
    # plt.plot(np.transpose(after_reweighting_total))
    # plt.show()   
    

## RV vs. time from the results of above RV computation

In [ ]:
rv_dev_stats = rv_dev_info.analyze_multiple_ccfs()
rv_dev_reweight_stats = rv_dev_reweight_info.analyze_multiple_ccfs()
rv_lev2_stats = rv_lev2_info.analyze_multiple_ccfs()
rv_lev2_reweight_stats = rv_lev2_reweight_info.analyze_multiple_ccfs()

In [ ]:
print(rv_dev_stats)

In [ ]:
print(rv_dev_reweight_stats)

In [ ]:
print(rv_lev2_stats)

In [ ]:
print(rv_lev2_reweight_stats)

In [ ]:
# comparing weighted and un-weighted result
rv_stats = {'dev': rv_dev_stats, 'dev_weighted': rv_dev_reweight_stats, 
            'lev2': rv_lev2_stats,
            'lev2_weighted': rv_lev2_reweight_stats}

plot_velocity_time(rv_stats, "weighting analysis", {'dev': "Cindy before", 
                                                    'dev_weighted': "Cindy: after",
                                                    'lev2': 'NEID: before',
                                                    'lev2_weighted': "NEID: after"}, 
                   {'dev': 'cyan', 'dev_weighted': 'blue', 'lev2': 'magenta',
                    'lev2_weighted': 'red'},
                    time_unit='day')

In [ ]:
rv_stats = {'dev': rv_dev_stats, 'dev_weighted': rv_dev_reweight_stats}

plot_velocity_time(rv_stats, "weighting analysis", {'dev': "Cindy before", 
                                                    'dev_weighted': "Cindy: after"}, 
                   {'dev': 'cyan', 'dev_weighted': 'blue'},
                    time_unit='day')

In [ ]:
# comparing weighted and un-weighted result
rv_stats = {'lev2': rv_lev2_stats,
            'lev2_weighted': rv_lev2_reweight_stats}

plot_velocity_time(rv_stats, "weighting analysis", {'lev2': 'NEID: before',
                                                    'lev2_weighted': "NEID: after"}, 
                   {'lev2': 'magenta',
                    'lev2_weighted': 'red'},
                    time_unit='day')